In [1]:
import sys
sys.path.insert(0, '/Users/carsoncook/Dev/CS445/Group_Project_cs445')
from extractFeatures import readDataFile
from label_map import soundTitlesA as titles
import model_helper as mh

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.optimizers import RMSprop

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn

#Data files
AUDIO_DATA      = "../data/audioData.csv"
IMAGE_DATA      = "../data/imageData.csv"
AUDIO_DATA_NORM = "../data/audioDataNormalized.csv"
IMAGE_DATA_NORM = "../data/imageDataNormalized.csv"

#Constants
FEATURES = 135
# INPUT_SHAPE = (FEATURES, 1)
ROWS = 1
COLS = 135
N_CLASSES = 50

# Hyper Parameters
H_UNITS = 256
BATCH_SIZE = 128
EPOCHS = 100

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
aud_X_train, aud_y_train, aud_X_test, aud_y_test = readDataFile(AUDIO_DATA)
# aud_X_train = aud_X_train.reshape(aud_X_train.shape[0], ROWS, COLS)
aud_X_train = aud_X_train.reshape(-1, ROWS, COLS)
aud_X_test = aud_X_test.reshape(-1, ROWS, COLS)
aud_X_train.shape

(1500, 1, 135)

LSTM RNN

One-Hot Encoding

In [3]:
aud_Y_train = keras.utils.to_categorical(aud_y_train, N_CLASSES)
aud_Y_test = keras.utils.to_categorical(aud_y_test, N_CLASSES)

In [4]:
# RNN LSTM
aud_model = mh.build_and_compile(h_units=H_UNITS, inp_shape=(ROWS, COLS), n_classes=N_CLASSES)
aud_model.fit(aud_X_train, aud_Y_train,
         batch_size=BATCH_SIZE,
         epochs=EPOCHS,
         verbose=0,
         validation_data=(aud_X_test, aud_Y_test))
score = aud_model.evaluate(aud_X_test, aud_Y_test, verbose=0)
print("Score: ", score)

Score:  [2.3794573583602907, 0.49800000023841856]


In [5]:
import seaborn as sb

In [6]:
preds = aud_model.predict_classes(aud_X_test)

In [7]:
pred_labs = np.array([titles[lab] for lab in preds])
true_labs = np.array([titles[y] for y in aud_y_test])

In [9]:
cm = confusion_matrix(aud_y_test, preds)
mh.make_heatmap(cm, 'Accuracy', 'heatmap.png')